In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
tf.__version__

'1.14.0'

In [6]:
def map_and_reversed_map(x):
    return (x, dict(zip(x.values(), x.keys())))

In [85]:
batch_size = 1

(emoji_index, index_emoji) = map_and_reversed_map({
    '🐀': 27,
    '🏠': 28,
    '🎩': 29,
    '🐈': 30,
    '✈': 31,
    '😣': 32
})

def index_to_char(i):
    return chr(i + 97 if (i != 26) else 32) if (i < 1000) else index_emoji[i]
def char_to_index(ch):
    return (ord(ch) - 97 if (ch != ' ') else 26) if (ord(ch) < 1000) else emoji_index[ch]

encoding_size = 33

In [103]:
def strings_to_categories(*strings):
    return [ list(keras.utils.to_categorical(list(map(char_to_index, list(string))), encoding_size)) for string in strings ]

In [108]:
x_train = strings_to_categories("rat  ") #, "house", "hat  ", "cat  ", "plane", "matt ")
y_train = strings_to_categories('🐀') #, '🏠', '🎩', '🐈', '✈', '😣')

print(y_train)

[[[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
      dtype=float32)]]]


In [107]:
model = keras.Sequential([
    layers.LSTM(1, input_shape=(None, encoding_size)),
])

model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=0.05),
    loss=keras.losses.categorical_crossentropy
)

def on_epoch_end(epoch, data):
    if epoch % 10 == 9:
        print("epoch", epoch)
        print("loss", data['loss'])
        
        # Generate text from the initial text ' h'
        texts = ['rt ', 'rats']
        for text in texts:
            x = [[list(keras.utils.to_categorical(list(map(char_to_index, list(text)))))]]
            y = model.predict(x)[0][-1]
            print(index_to_char(y.argmax()))        
        
model.fit(
    x_train, 
    y_train, 
    # batch_size=batch_size, 
    epochs=500, 
    verbose=False,
    callbacks=[keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)]
)

InvalidArgumentError: Expected multiples argument to be a vector of length 1 but got length 2
	 [[{{node RMSprop_20/gradients/loss_20/lstm_20_loss/Sum_2_grad/Tile}}]]